In [1]:
import numpy as np
from PIL import Image
# from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# def multiprocessing(func, args, workers):
#     with ProcessPoolExecutor(workers) as ex:
#         res = ex.map(func, args)
#     return list(res)

In [3]:
a, b, step = -2, 2, 0.001 # start, stop, and step values for the 'window'.
# N = np.arange(a, b+step, step)
w, h = 2000, 2000
X = np.linspace(-1.5, 1.5, w)
Y = np.linspace(-1.5, 1.5, h) # if the width is 50% larger than height, than the interval [start, stop] should be 50% larger and the number of steps should be 50% greater.
# N_2 = 

def cartesian_product(x, y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

# c_0 = cartesian_product(N, N)
c_0 = cartesian_product(X, Y)

# print(N)
print(c_0)

# convert the 2D array into an array of complex numbers. 
c_0_i = 1j*c_0[...,1] # build the complex components
c_0_i += c_0[...,0] # build the real components

c_0_i += -1

print(c_0_i)

[[-1.5        -1.5       ]
 [-1.49849925 -1.5       ]
 [-1.4969985  -1.5       ]
 ...
 [ 1.4969985   1.5       ]
 [ 1.49849925  1.5       ]
 [ 1.5         1.5       ]]
[-2.5       -1.5j -2.49849925-1.5j -2.4969985 -1.5j ...  0.4969985 +1.5j
  0.49849925+1.5j  0.5       +1.5j]


In [4]:
print(c_0_i.shape)

(4000000,)


In [28]:
# MAX_ITERATIONS = 1
# MAX_VALUE = 4

# def mandelbrot(loc):
# #     print(loc, loc.shape, type(loc))
#     iteration = 0
#     z = 1j * 0
#     while iteration < MAX_ITERATIONS and abs(z) < MAX_VALUE and abs(z) > 0:
#         z = z**2 + loc
#         iteration += 1
#     return iteration

# z = multiprocessing(mandelbrot, c_0_i, 8)

In [5]:
# z_(n+1) = z_(n)^2 + c 
# c = some complex number (in the complex plane)
# z_0 = 0
# iteration applied repeatedly. 

z = 1j * np.zeros(len(c_0)) # z_0 = 0
history = np.expand_dims(z, 0) # use history to make a color gradient 
print(z.shape, history.shape)

(4000000,) (1, 4000000)


In [6]:
for n in range(40):
    z = z**2 + c_0_i
    history = np.concatenate((history, np.expand_dims(z, 0)), 0)
    
# bounded = z[abs(z)<1]
# print(len(c_0_i), len(bounded))
print(max(abs(z)))

# other ideas: Use quaternions in the formula, visualize in 4D as a video, or reduce to 3 values and visualize in 3D!

# IDEA: Parallelize across pixels to determine which iteration the pixel exceeds a cutoff value. 
# have a map across all the pixels, then calculate which iteration each pixel exceeds some cutoff. 

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in square
  


NameError: name 'bounded' is not defined

In [7]:
print(history.shape)
THRESH = 10
argmins = np.argmin(abs(abs(history) - THRESH), 0)
print(argmins.shape, argmins[:100])

# argmaxes = np.argmax(history, 0)

# print(argmaxes.shape, argmaxes[:10])


(41, 4000000)


/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in absolute
  This is separate from the ipykernel package so we can avoid doing imports until


(4000000,) [12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12]


In [33]:
argmins_list = list(argmins)
counts = {i:argmins_list.count(i) for i in list(set(argmins_list))}

In [34]:
print(counts)

{1: 113054, 2: 108390, 3: 63480, 4: 19430, 5: 15896, 6: 11592, 7: 6216, 8: 4944, 9: 2939, 10: 6028, 11: 1129154, 12: 1357974, 13: 483961, 14: 220363, 15: 118262, 16: 73888, 17: 47947, 18: 35139, 19: 25787, 20: 20076, 21: 15276, 22: 13364, 23: 10064, 24: 8996, 25: 7724, 26: 6780, 27: 5450, 28: 5556, 29: 4188, 30: 4058, 31: 3633, 32: 3313, 33: 3078, 34: 2980, 35: 2249, 36: 2397, 37: 2268, 38: 2254, 39: 1690, 40: 2080, 41: 1494, 42: 1698, 43: 1620, 44: 1503, 45: 1075, 46: 1442, 47: 1122, 48: 1296, 49: 1280, 50: 1238, 51: 1214, 52: 1734, 53: 1409, 54: 1377, 55: 1284, 56: 1384, 57: 1116, 58: 1590, 59: 1062, 60: 2144}


In [10]:
import matplotlib as plot

# map iteration numbers to hues.
def color_map(argmins, z):
    # IDEA: Use black for values below some cutoff, and hues for values above that. 
    h = (abs(argmins) % (41) + 240) / 360.0 # calculate hue.
    
    
#     h[np.isnan(c)] = 90 / 360 
    
    hsv = np.column_stack((h, np.ones(len(h)), np.zeros(len(h)) + 0.5))
    hsv[abs(z)<2] = np.zeros(3)
    rgb = plot.colors.hsv_to_rgb(hsv)
#     rgb[np.isnan(c)] = np.zeros(3)
    rgb_int = (rgb * 256).astype(np.uint8) 
    return rgb_int

z_rgb = color_map(argmins, z)
print(z_rgb)
# rearrange the 1D array of hue values to 2D.
# s = int(len(z_rgb)**(1/2))
img = Image.fromarray(z_rgb.reshape(w,h,3))
img.show()

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in less
  if sys.path[0] == '':


[[ 25   0 128]
 [ 25   0 128]
 [ 25   0 128]
 ...
 [ 27   0 128]
 [ 27   0 128]
 [ 27   0 128]]


In [12]:
import matplotlib as plot


# Create high contrast between positive and negative space of the fractal
# pos_space = [<primary_colors...>] # cardnality of set determines visual complexity (could be function)
# Instead of 'h = (360 - np.log(abs(c)) % (180)) / (360)' - 
# (c) => return 'position in pos_space 

# neg_space = [<secondary_colors...>] # cardnality of set determines visual complexity (could be function)
# Instead of 'h[np.isnan(c)] = 90 / 360 ' - 
# (c) => return 'position in neg_space

# map absolute values to hues.
def color_map(c):
    # IDEA: Use black for values below some cutoff, and hues for values above that. 
    h = (360 - np.log(abs(c)) % (180)) / (360) # calculate hue.
#     h[c<1] = 0
    
    h[np.isnan(c)] = 90 / 360 
    
    hsv = np.column_stack((h, np.ones(len(h)), np.zeros(len(h)) + 0.5))
    rgb = plot.colors.hsv_to_rgb(hsv)
#     rgb[np.isnan(c)] = np.zeros(3)
    rgb_int = (rgb * 256).astype(np.uint8) 
    return rgb_int

z_rgb = color_map(z)
print(z_rgb)
# rearrange the 1D array of hue values to 2D.
# s = int(len(z_rgb)**(1/2))
img = Image.fromarray(z_rgb.reshape(w,h,3))
img.show()

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


[[ 64 128   0]
 [ 64 128   0]
 [ 64 128   0]
 ...
 [ 64 128   0]
 [ 64 128   0]
 [ 64 128   0]]


In [26]:
# data = np.zeros( (1024,1024,3), dtype=np.uint8 )

# data[512,512] = [254,0,0]       # Makes the middle pixel red
# data[512,513] = [0,0,255]       # Makes the next pixel blue

# img = Image.fromarray( data )       # Create a PIL image
# img.show()    
